# Tester notebook

This notebook aims to create enviroment to simply test strategies developed in Stocks & Loss system for python

In [3]:
# imports 
from scraper import stock_daily
from analyzer import Analyzer
import numpy as np
import pandas as pd
from IPython.display import display
import datetime as dt
import indicators as ind
import warnings

warnings.filterwarnings('ignore')

In [13]:
dt.datetime.today().strftime("%d-%m-%Y at %H:%M:%S")

'04-12-2021 at 17:42:01'

## Load stocks across industries

In [ ]:
xls = pd.ExcelFile('TickerPerIndexFund.xls')
fundTickers = pd.DataFrame()
Tickers = pd.DataFrame()
fundTickers["Fund/sector name"] = xls.sheet_names
# Tickers = pd.DataFrame(,columns=funTickers["Index fund name"])
for i in range(len(xls.sheet_names)):
    sheet1 = pd.read_excel(xls, i)
    fundTickers.loc[i,["Fund/sector ticker"]] = sheet1.columns[1]
    Tickers[fundTickers.loc[i,["Fund/sector name"]]] = sheet1[sheet1.columns[1]][1:]
pd.set_option('display.expand_frame_repr', False)    
display(Tickers)

## Iterate over each industry

In [ ]:
# I have the initialization here on purpose, to measure its impact
col = ["Ticker", "Trades", "Good trades", "Profit[%]", "Profit[$]", "Hold[%]"]
results = [None] * Tickers.columns.shape[0]
for k in range(Tickers.columns.shape[0]):
    # when I tried to initialize results as [dataframe] * 10 it created list of identical dataframes and when i changed one, it changed them all. Interesting
    results[k] = pd.DataFrame(data=np.zeros([Tickers.shape[0], len(col)]),columns=col)

In [ ]:
# iterate over columns
i = 0 
for label, tic in Tickers.iteritems():
    print(label)
    # iterate over items in sectors / indexes
    for j, val in tic.iteritems():
        # j starts at 1, instead of usual 0 => reason why there is -1 everywhere
        # analyze stock
        try:
            stock = stock_daily(val, save=False)
            trades = Analyzer(ticker=val,data=stock.data)
            
            # STRATEGY
            # building buy signals
            # defining uptrend
            up = (ind.ema(stock.data, w=13)["EMA"] > ind.ema(stock.data, w=50)["EMA"]).to_numpy()
            # buy signals in uptrend
            bsu = np.logical_and(trades.mb_stoch(period=21, sk=3, sd=5, treshold=20), up)
            # buy signals in downtrend
            bsd = np.logical_and(trades.mb_stoch(period=21, sk=3, sd=5, treshold=20), np.logical_not(up))
            # finalize buy signals
            bsf = np.logical_or(bsu, bsd)

            # throw exception if buy signal is empty
            if bsf.sum() == 0:
                raise RuntimeError("No buy signals were generated by this strategy.")
            # for N buy signals, create N sell vectors where Nth column will be the sell signal for Nth buy signal, 
            # some sell methods that are independent will have copied columns

            ss = trades.ms_smacross(fast=5, slow=10)

            ssf = np.repeat([ss,], repeats=np.sum(bsf), axis=0)
            # sort signals
            ss = trades.signalSorter(bsf,ssf,rb=False) 
            
            trades.strategy(ss,stopLoss=False,stopLossValue=0.05,profitTaker=False,profitTakerValue=0.05)
            ts = trades.profit(capitalForEachTrade=400,comission=2)
            # save data into dataframe
            results[i].loc[j-1,["Ticker"]] = val
            results[i].loc[j-1,["Trades"]] = ts.shape[0]
            results[i].loc[j-1,["Good trades"]] = ts["Good trade?"].sum()
            results[i].loc[j-1,["Profit[%]"]] = ts["Profit[%]"].sum()
            results[i].loc[j-1,["Profit[$]"]] = ts["Profit[$]"].sum()
            results[i].loc[j-1,["Hold[%]"]] = 100*((trades.data["Close"].iloc[-1]-trades.data["Close"].iloc[0])/trades.data["Close"].iloc[0])
        except:
            results[i].loc[j-1,["Ticker"]] = val
    i += 1

In [ ]:
for k in range(Tickers.columns.shape[0]):
    print(Tickers.columns[k],': profit from trading [$] = ',np.round(np.sum(results[k]["Profit[$]"]),1),' after ',np.round(np.sum(results[k]["Trades"]),1),' trades.')


In [ ]:
display(results[0])